# GLTF 格式教學 Scene & Node 篇

<a href="https://colab.research.google.com/github/CSP-GD/notes/blob/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/scene-node/scene-node.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

[`Open in observablehq`](https://observablehq.com/@toonnyy8/gltf-scene-node/9)

![圖 1. scenes, nodes \[1\]](https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/scene-node/gltfOverview-2.0.0b-scene-node.png)

圖 1. scenes, nodes \[1\]

## 簡介

### Scene 簡介

glTF 中可以儲存多個場景，而每個場景都會存在「scenes」屬性中，  
由「scene」屬性指向預設的場景，而「scenes」內會有「nodes」，  
表示每個場景所擁有的。

### Node 簡介

每個 node 都可以當成是場景中的一個物件，  
每個物件具有自己的名稱「name」，  
這些物件有可能是「mesh」、「skin」、「camera」、「light」等等，  
而物件之間還具有親子關係，並利用「children」用來記錄哪些 node 是自己的子代，  
最後還有「matrix」來記錄此物件的初始姿態。  
> ps.「matrix」可拆分成  
> 1. 代表偏移的「translation」
> 2. 代表旋轉的「rotation」
> 3. 代表縮放的「scale」

## 正式開始

In [0]:
from IPython.display import JSON

### 載入 glTF_tools

In [1]:
!wget https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb -O gltf-tools.ipynb
%run ./gltf-tools.ipynb

--2020-04-07 14:03:14--  https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb [following]
--2020-04-07 14:03:15--  https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3699 (3.6K) [text/plain]
Saving to: ‘gltf-tools.ipynb’

gltf-tools.ipynb    100%[==============

## 載入檔案

In [2]:
!wget https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/scene-node/cube-channel.glb -O cube-channel.glb
glb_file = open('./cube-channel.glb', 'rb')
glb_bytes = glb_file.read()
model = glTF_tools.glb_loader(glb_bytes)

--2020-04-07 14:03:22--  https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/scene-node/cube-channel.glb
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/scene-node/cube-channel.glb [following]
--2020-04-07 14:03:23--  https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/scene-node/cube-channel.glb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4544 (4.4K) [application/octet-stream]
Saving to: ‘cube-ch

### 查看場景

In [9]:
print("預設場景為 scenes[{}]".format(model["scene"]))

預設場景為 scenes[0]


In [51]:
scene = model["scenes"][model["scene"]]

import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="width:100%;"></div>'.format(self.uuid), raw=True)
        display_html("""
        <script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
        <script>
            document.getElementById('%s').appendChild(renderjson(%s))
        </script>
        """ % (self.uuid, self.json_str), raw=True)
display(RenderJSON(scene))

本場景中只有一個 node : nodes\[2\]

In [13]:
node = model["nodes"][scene["nodes"][0]]

node

{'children': [1], 'mesh': 2, 'name': 'Cube'}

從其中內容可以知道
1. 有子代，子代為 nodes\[1\]
2. 此 node 代表的物件為 mesh，並指向 meshs\[2\]
3. 此物件的名字為 Cube
4. 沒有另外設定初始姿態

查看子代 nodes\[1\]

In [18]:
child_node = model["nodes"][node["children"][0]]

child_node

{'children': [0], 'mesh': 1, 'name': 'Cube.001', 'translation': [3, 0, 0]}

從其中內容可以知道
1. 有子代，子代為 nodes\[0\]
2. 此 node 代表的物件為 mesh，並指向 meshs\[1\]
3. 此物件的名字為 Cube.001
4. 初始姿態向 X 軸正向位移 3 單位

查看子代的子代 nodes\[0\]

In [20]:
grandchildren_node = model["nodes"][child_node["children"][0]]

grandchildren_node

{'mesh': 0, 'name': 'Cube.002', 'translation': [3, 0, 0]}

從其中內容可以知道
1. 並沒有子代
2. 此 node 代表的物件為 mesh，並指向 meshs\[0\]
3. 此物件的名字為 Cube.002
4. 初始姿態向 X 軸正向位移 3 單位

另外要注意，每個物件的姿態矩陣都會對子代產生作用，  
因此 grandchildrenNode 實際上受到 childNode 的影響後，  
總共會向 X 軸正向位移 6 單位。

## 參考

1. https://github.com/KhronosGroup/glTF